In [235]:
import pandas as pd
from numpy import unique

from src.OtherTasks.ML.PreDeal_Tools import Del_deletion_data, Record_usable_cols, Deal_sorted_Ydata, convert_to_num, \
    get_final_useablecols, all_ydata, delAndGetCols, get_merge_tabledata, deal_verify_ydata, get_former_Ydata
import numpy as np
# 这个pandas处理数据效果不太好 建议用numpy
thedata = pd.read_excel(io='Data613.xlsx', sheet_name='数据标准化-建模用')
Preddata = pd.read_excel(io='Data613.xlsx', sheet_name='数据标准化-预测用')

# 获取数据表对象
thedata = np.array(thedata)
Preddata = np.array(Preddata)
# 将表的内容 7个数据表 挨个处理

In [236]:
# 获取源建模的X数据 传入的对象是获取的excel对象
FormerTwo_data, FormerThree_data, FormerFour_data, FormerFive_data, FormerSix_data, FormerSeven_data = get_merge_tabledata(thedata)
# 获取预测的X数据
Verify_TABLE_TWO, Verify_TABLE_THREE, Verify_TABLE_FOUR, Verify_TABLE_FIVE, Verify_TABLE_SIX, Verify_TABLE_SEVEN = get_merge_tabledata(Preddata)

# 删除缺失值过多的行，并保存del_cols
tempFormerTwo_data = delAndGetCols(FormerThree_data)
base_Xdata = tempFormerTwo_data[0]
del_cols = tempFormerTwo_data[1]
# 用del_cols删除验证集X不需要的列
Verify_Xdata = np.delete(Verify_TABLE_THREE, del_cols, axis=1)

tempFormerTwo_data

(array([[-1, 1, 2.2134, ..., -0.74466, 1.60837, '0'],
        [-1, 1, 2.2134, ..., -0.74466, 0.94436, '0'],
        [-1, 1, 0.72748, ..., -0.74466, 0.94436, '0'],
        ...,
        [2, -1, -0.01548, ..., 0.55961, 0.61235, '1'],
        [2, -1, -0.01548, ..., 0.55961, 0.94436, '1'],
        [2, -1, -0.01548, ..., 0.55961, 0.28034, '1']], dtype=object),
 [2, 3, 5, 7, 8, 22])

In [237]:
# 获取源建模的数据Y_data
Original_TableTwoYdata, Original_TableThreeYdata, Original_TableFiveYdata, Original_TableSixYdata, Original_TableSevenYdata=get_former_Ydata(thedata)
# 获取预测数据的Y_data
Predict_TableTwoYdata, Predict_TableThreeYdata, Predict_TableFiveYdata, Predict_TableSixYdata, Predict_TableSevenYdata=get_former_Ydata(Preddata)

In [238]:
# 将Y取出来，然后取出缺失值过多的列

# 获取可用的列
final_cols = get_final_useablecols(Original_TableThreeYdata, Predict_TableThreeYdata)
final_cols

[8, 9, 10, 11, 12, 13, 14, 15, 16]

In [239]:
# 获取可以用的Y_data
Original_YdataList, Pred_YdataList = all_ydata(final_cols, Original_TableTwoYdata, Predict_TableTwoYdata)
# todo 到时候遍历这个列表，然后依次计算,依次去建模预测
# 建模的Y_data
Modeling_Y_data = Original_YdataList[1]
# 预测的Y_data
Pred_Y_data = Pred_YdataList[1]

In [ ]:
# for index in range(len(Modeling_Y_data)):
#     start_pred()

In [240]:
# 先删除缺失值，然后再去将Y_data分类
# 删除缺失值过多的行
# 处理建模的X_data,Y_data
temp_OriginalXdata = np.column_stack((base_Xdata, Modeling_Y_data))
# 删除缺失的行
temp_OriginalXdata = Del_deletion_data(temp_OriginalXdata, 0)
X_data = temp_OriginalXdata[:, :-1]
Y_data = temp_OriginalXdata[:, -1]
# 处理验证的X_data,Y_data
# 合并数据
temp_verifydata = np.column_stack((Verify_Xdata, Pred_Y_data))
# 删除缺失的行
temp_verifydata = Del_deletion_data(temp_verifydata, 0)
# 获取预测数据的X_data，Y_data
X_verify_data = temp_verifydata[:, :-1]
Y_verify_data = temp_verifydata[:, -1]

In [241]:
# 检验一下是否将所有的空值删除完毕，这个地方是用来做验证的 返回为空列表则删除成功！
list(set(np.where(np.isnan(temp_OriginalXdata.astype(float)) == True)[0].tolist())), list(set(np.where(np.isnan(temp_verifydata.astype(float)) == True)[0].tolist()))

([], [])

In [242]:
# 将建模Y_data分类(-1,0,1)，并且取出边界值
Y_data,Y_data_boundsMin,Y_data_boundsMax=Deal_sorted_Ydata(Y_data)
# 获取预测数据的Y_data Verify_Ydata = Pred_Y_data
Verify_Ydata = deal_verify_ydata(Y_verify_data, Y_data_boundsMin, Y_data_boundsMax)
# 查看是否替换成功
[unique(Y_data),Y_data_boundsMin,Y_data_boundsMax],np.unique(Verify_Ydata)

([array([-1, 0, 1], dtype=object), -2.24214, 0.77777],
 array([-1, 0, 1], dtype=object))

In [243]:
# 切分训练数据和测试数据
from sklearn.model_selection import train_test_split
## 30%测试数据，70%训练数据，stratify=y表示训练数据和测试数据具有相同的类别比例
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=2, stratify=Y_data)

In [244]:
from sklearn.metrics import accuracy_score
# 开始训练模型
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# 获取模型  随机森林
Rfc = RandomForestClassifier()
Rfc.fit(X_train, y_train.astype(float))
Rfc_pred = Rfc.predict(X_test)
Rfc_acc= accuracy_score(y_test.astype(float), Rfc_pred)
print('Rfc_acc准确率:', Rfc_acc)

# 梯度提升树
Gbc = GradientBoostingClassifier()
Gbc.fit(X_train, y_train.astype(float))
Gbc_pred = Gbc.predict(X_test)
Gbc_acc=accuracy_score(y_test.astype(float), Gbc_pred)
print('Gbc_acc准确率:',Gbc_acc)

# 使用SVM模型
from sklearn.svm import SVC
Svc = SVC()
Svc.fit(X_train, y_train.astype(float))
Svc_pred = Svc.predict(X_test)
Svc_acc = accuracy_score(y_test.astype(float), Svc_pred)
print('SVC准确率：', Svc_acc)

# 使用KNN模型
from sklearn.neighbors import KNeighborsClassifier
Knc = KNeighborsClassifier()
Knc.fit(X_train, y_train.astype(float))
Knc_pred = Knc.predict(X_test)
Knc_acc = accuracy_score(y_test.astype(float), Knc_pred)
print('KNC准确率：', Knc_acc)

# 使用AdaBoost模型
from sklearn.ensemble import AdaBoostClassifier
Abc = AdaBoostClassifier()
Abc.fit(X_train, y_train.astype(float))
Abc_pred = Abc.predict(X_test)
Abc_acc = accuracy_score(y_test.astype(float), Abc_pred)
print('ABC准确率：', Abc_acc)

# 使用DNN模型
from sklearn.neural_network import MLPClassifier
Dnn = MLPClassifier()
Dnn.fit(X_train, y_train.astype(float))
Dnn_pred = Dnn.predict(X_test)
Dnn_acc = accuracy_score(y_test.astype(float), Dnn_pred)
print('DNN准确率：', Dnn_acc)

# 使用卷积神经网络模型
from sklearn.neural_network import MLPClassifier
CNN = MLPClassifier()
CNN.fit(X_train, y_train.astype(float))
CNN_pred = CNN.predict(X_test)
CNN_acc = accuracy_score(y_test.astype(float), CNN_pred)
print('CNN准确率：', CNN_acc)

# 使用LDA模型
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
Lda = LinearDiscriminantAnalysis()
Lda.fit(X_train, y_train.astype(float))
Lda_pred = Lda.predict(X_test)
Lda_acc = accuracy_score(y_test.astype(float), Lda_pred)
print('LDA准确率：', Lda_acc)

# 使用QDA模型
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
Qda = QuadraticDiscriminantAnalysis()
Qda.fit(X_train, y_train.astype(float))
Qda_pred = Qda.predict(X_test)
Qda_acc = accuracy_score(y_test.astype(float), Qda_pred)
print('QDA准确率：', Qda_acc)


Rfc_acc准确率: 0.9
Gbc_acc准确率: 0.9
SVC准确率： 0.9
KNC准确率： 0.88
ABC准确率： 0.8


C:\All_Softwares\Develop_Tools\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


DNN准确率： 0.9
CNN准确率： 0.9
LDA准确率： 0.88
QDA准确率： 0.9


C:\All_Softwares\Develop_Tools\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\All_Softwares\Develop_Tools\Python310\lib\site-packages\sklearn\discriminant_analysis.py:878: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [245]:
k = 0
for i in range(Rfc_pred.shape[0]):
    if y_test[i] == Rfc_pred[i]:
        k += 1
k / y_test.shape[0]

0.9

In [246]:

# 预测数据
Rfc_pred = Rfc.predict(X_verify_data)
Gbc_pred = Gbc.predict(X_verify_data)
Svc_pred = Svc.predict(X_verify_data)
Knc_pred = Knc.predict(X_verify_data)
Abc_pred = Abc.predict(X_verify_data)
Dnn_pred = Dnn.predict(X_verify_data)
CNN_pred = CNN.predict(X_verify_data)
Lda_pred = Lda.predict(X_verify_data)
Qda_pred = Qda.predict(X_verify_data)
# 计算准确率
verify_y1 = accuracy_score(Y_verify_data.astype(float), Rfc_pred)
verify_y2 = accuracy_score(Y_verify_data.astype(float), Gbc_pred)
verify_y3 = accuracy_score(Y_verify_data.astype(float), Svc_pred)
verify_y4 = accuracy_score(Y_verify_data.astype(float), Knc_pred)
verify_y5 = accuracy_score(Y_verify_data.astype(float), Abc_pred)
verify_y6 = accuracy_score(Y_verify_data.astype(float), Dnn_pred)
verify_y7 = accuracy_score(Y_verify_data.astype(float), CNN_pred)
verify_y8 = accuracy_score(Y_verify_data.astype(float), Lda_pred)
verify_y9 = accuracy_score(Y_verify_data.astype(float), Qda_pred)

print('Gbc', verify_y1)
print('Rfc', verify_y2)
print('Svc', verify_y3)
print('Knc', verify_y4)
print('Abc', verify_y5)
print('Dnn', verify_y6)
print('CNN', verify_y7)
print('Lda', verify_y8)
print('Qda', verify_y9)


Gbc 0.7386363636363636
Rfc 0.7386363636363636
Svc 0.7386363636363636
Knc 0.7386363636363636
Abc 0.3693181818181818
Dnn 0.7386363636363636
CNN 0.7386363636363636
Lda 0.7329545454545454
Qda 0.7386363636363636
